## Task planning & Goal: 
### Metrics/hypothesis for generating data insights
###### 1. Customer 
- Business model: conversion, retention and generate revenue, customer turnover, LTV
- Customer churn, repurchase rate to measure "stickness"(retention) and loyalty/-purchase behaviors of customer
- ==> Build the function that will generate the "cohorts"
- New Customer acquisition(time base: monthly)
- is discount promotion increase sales or new customers acquisition(H0)

###### 2. Product: 4P rule
- Products-> Pricing -> Place(online, if Omi-channel?-> offline: demographic city/region)

###### 3. ML
- Timeseries analysis: Sales and Profits
- Sales target section(model: linear regression, using Sklearn), to show confidence for the future sales. 
- Anomaly detetcion: For e-commerce there are always some fake purchases
- Cluserting, TODO

### Workflow
- Data analysis for cleaning and aggregation
- Build Small Database (MySQL)
- Connect Database with Apache Superset and deploy
- Optional use Tableau 
- Predictive analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# build helper functions
FILE_PATH= "Data Analyst Case Chaoly.xlsx"
def get_data(base_file: str, tab_nb: int, order_date: str=None, ship_date: str=None)-> pd.DataFrame:
    return pd.read_excel(base_file,
                         tab_nb,
                        parse_date = ["order_date","ship_date"],
                        infer_datetime_format = True)

In [2]:
# read dataset
ListOfOrders = get_data(FILE_PATH, 0, "Order Date", "Ship Date")
OrderBreakdown = get_data(FILE_PATH, 1)
SalesTargets = get_data(FILE_PATH, 2)

#### EDA 
- check is any Null values, if then dropna

- check row-wise duplicated ->remove 

In [3]:
def isnull(df):
    return df.isnull().any()

In [4]:
isnull(ListOfOrders)

Order ID         False
Order Date       False
Customer Name    False
City             False
Country          False
Region           False
Segment          False
Ship Date        False
Ship Mode        False
State            False
dtype: bool

In [5]:
isnull(OrderBreakdown)
isnull(SalesTargets) 

Month of Order Date    False
Category               False
Target                 False
dtype: bool

In [6]:
ListOfOrders.duplicated().any() 

False

In [7]:
OrderBreakdown.duplicated().any()
# OrderBreakdown.columns

True

In [8]:
OrderBreakdown[OrderBreakdown.duplicated("Order ID")]

,Order ID,Product Name,Discount,Sales,Profit,Quantity,Category,Sub-Category
4,BN-2011-2819714,"Eldon Folders, Single Width",0.50,17,-1,2,Office Supplies,Storage
6,AZ-2011-617423,"Sanford Canvas, Fluorescent",0.00,207,77,4,Office Supplies,Art
8,AZ-2011-2918397,"Accos Thumb Tacks, Assorted Sizes",0.00,33,2,3,Office Supplies,Fasteners
9,AZ-2011-2918397,"Smead Lockers, Industrial",0.10,716,143,4,Office Supplies,Storage
11,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.50,116,-56,5,Office Supplies,Art
13,AZ-2011-7053593,"Binney & Smith Canvas, Blue",0.00,103,20,2,Office Supplies,Art
16,AZ-2011-4827146,"Smead Trays, Single Width",0.00,97,31,2,Office Supplies,Storage
17,AZ-2011-6439906,"Novimex File Folder Labels, Alphabetical",0.00,40,6,5,Office Supplies,Labels
22,AZ-2011-5702370,"Binney & Smith Canvas, Blue",0.00,257,49,5,Office Supplies,Art
23,AZ-2011-5702370,"Ibico Index Tab, Clear",0.00,17,6,2,Office Supplies,Binders


In [9]:
SalesTargets.duplicated().any() 

False

In [10]:
# ListOfOrders.tail() 
ListOfOrders[ListOfOrders["Order ID"]=="BN-2014-4140795"] 

,Order ID,Order Date,Customer Name,City,Country,Region,Segment,Ship Date,Ship Mode,State
4116,BN-2014-4140795,2014-12-31,Daniel Hamilton,Eindhoven,Netherlands,Central,Home Office,2015-01-05,Economy Plus,North Brabant


In [11]:
OrderBreakdown[OrderBreakdown["Order ID"]=="BN-2014-4140795"] 

,Order ID,Product Name,Discount,Sales,Profit,Quantity,Category,Sub-Category
8043,BN-2014-4140795,"BIC Pencil Sharpener, Fluorescent",0.5,30,-10,2,Office Supplies,Art
8044,BN-2014-4140795,"Avery Binder Covers, Recycled",0.5,23,-6,4,Office Supplies,Binders
8045,BN-2014-4140795,"StarTech Phone, Red",0.5,108,-19,3,Technology,Machines


In [12]:
# For each main category=>sub-category, monthly sales target below. 
SalesTargets.head() 

,Month of Order Date,Category,Target
0,2011-01-01,Furniture,10000
1,2011-02-01,Furniture,10100
2,2011-03-01,Furniture,10300
3,2011-04-01,Furniture,10400
4,2011-05-01,Furniture,10500


In [13]:
SalesTargets.Category.unique() 

array(['Furniture', 'Office Supplies', 'Technology'], dtype=object)

In [14]:
# merge ListofOders and OrdersBrokendown tables
# one tick: use copy() to not modify the original data sets, so that to trackback and use the raw fact data again.
FullOrderList= ListOfOrders.merge(OrderBreakdown, on="Order ID", how="left")
FullOrderList.tail()

,Order ID,Order Date,Customer Name,City,Country,Region,Segment,Ship Date,Ship Mode,State,Product Name,Discount,Sales,Profit,Quantity,Category,Sub-Category
8042,AZ-2014-1412225,2014-12-31,Leon Barnes,Worcester,United Kingdom,North,Consumer,2015-01-01,Priority,England,"Fellowes Shelving, Single Width",0.0,289,75,5,Office Supplies,Storage
8043,AZ-2014-7604524,2014-12-31,Rebecca Chamberlain,Hamburg,Germany,Central,Home Office,2015-01-04,Economy,Hamburg,"Wilson Jones Index Tab, Economy",0.0,32,8,5,Office Supplies,Binders
8044,BN-2014-4140795,2014-12-31,Daniel Hamilton,Eindhoven,Netherlands,Central,Home Office,2015-01-05,Economy Plus,North Brabant,"BIC Pencil Sharpener, Fluorescent",0.5,30,-10,2,Office Supplies,Art
8045,BN-2014-4140795,2014-12-31,Daniel Hamilton,Eindhoven,Netherlands,Central,Home Office,2015-01-05,Economy Plus,North Brabant,"Avery Binder Covers, Recycled",0.5,23,-6,4,Office Supplies,Binders
8046,BN-2014-4140795,2014-12-31,Daniel Hamilton,Eindhoven,Netherlands,Central,Home Office,2015-01-05,Economy Plus,North Brabant,"StarTech Phone, Red",0.5,108,-19,3,Technology,Machines


In [15]:
OrderLists= FullOrderList.copy()
# OrderLists.columns

In [16]:
OrderLists.groupby(["Segment"])["Sales"].sum()

Segment
Consumer       1236282
Corporate       738137
Home Office     374063
Name: Sales, dtype: int64

In [17]:
# transform the date for the %y-m format 
# use this compare to sales target! 
OrderLists["Order Month"]= OrderLists["Order Date"].apply(lambda d: d.strftime("%Y-%m"))
# get actual monthly sales, categorized by "main Category"
actual_monthly_sales = OrderLists.groupby(["Order Month","Category"])["Sales", "Profit"].sum().reset_index() 
actual_monthly_sales.tail() 

,Order Month,Category,Sales,Profit
139,2014-11,Office Supplies,33305,4090
140,2014-11,Technology,29909,2156
141,2014-12,Furniture,18843,2388
142,2014-12,Office Supplies,30133,2511
143,2014-12,Technology,34112,4303


In [18]:
# rename SalesTargets field
SalesTargets = SalesTargets.rename(columns={"Month of Order Date":"Order Month"})
SalesTargets["Order Month"] = SalesTargets["Order Month"].apply(lambda d: d.strftime("%Y-%m"))
SalesCombined = SalesTargets.merge(actual_monthly_sales, on="Order Month", how="left")
SalesCombined["Target_achieved"]= np.where((SalesCombined["Sales"] > SalesCombined["Target"]), "Achieved","Not Achieved")
SalesCombined.tail() 

,Order Month,Category_x,Target,Category_y,Sales,Profit,Target_achieved
427,2014-11,Technology,23000,Office Supplies,33305,4090,Achieved
428,2014-11,Technology,23000,Technology,29909,2156,Achieved
429,2014-12,Technology,23000,Furniture,18843,2388,Not Achieved
430,2014-12,Technology,23000,Office Supplies,30133,2511,Achieved
431,2014-12,Technology,23000,Technology,34112,4303,Achieved


In [19]:
target_achieve_rate = sum(SalesCombined["Target_achieved"]=="Achieved")/SalesCombined["Target_achieved"].count() 
print(f"Acieved rate is {100 * target_achieve_rate:.2f}%")

Acieved rate is 55.09%


In [20]:
ListOfOrders.describe()

,Order ID,Order Date,Customer Name,City,Country,Region,Segment,Ship Date,Ship Mode,State
count,4117,4117,4117,4117,4117,4117,4117,4117,4117,4117
unique,4117,1214,792,999,15,3,3,1311,4,127
top,AZ-2012-5204554,2014-09-16 00:00:00,Jose Gambino,London,France,Central,Consumer,2014-09-22 00:00:00,Economy,England
freq,1,13,13,113,991,2234,2132,13,2445,624
first,NaN,2011-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,2011-01-05 00:00:00,NaN,NaN
last,NaN,2014-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,2015-01-05 00:00:00,NaN,NaN


In [21]:
"""get customer order rate if more than once-> order month"""
orders_count = ListOfOrders.groupby(["Customer Name"])["Order ID"].nunique()
repeated_orders = np.sum(orders_count > 1) / ListOfOrders['Customer Name'].nunique()
print(f'{100 * repeated_orders:.2f}% of customers repeated their orders.')

96.09% of customers repeated their orders.


In [22]:
ListOfOrders['OrderPeriod'] = ListOfOrders['Order Date'].apply(lambda x: x.strftime('%Y-%m'))
ListOfOrders.head() 
# calculate the cohort 
# ListOfOrders["Cohort"]= ???

,Order ID,Order Date,Customer Name,City,Country,Region,Segment,Ship Date,Ship Mode,State,OrderPeriod
0,BN-2011-7407039,2011-01-01,Ruby Patel,Stockholm,Sweden,North,Home Office,2011-01-05,Economy Plus,Stockholm,2011-01
1,AZ-2011-9050313,2011-01-03,Summer Hayward,Southport,United Kingdom,North,Consumer,2011-01-07,Economy,England,2011-01
2,AZ-2011-6674300,2011-01-04,Devin Huddleston,Valence,France,Central,Consumer,2011-01-08,Economy,Auvergne-Rhône-Alpes,2011-01
3,BN-2011-2819714,2011-01-04,Mary Parker,Birmingham,United Kingdom,North,Corporate,2011-01-09,Economy,England,2011-01
4,AZ-2011-617423,2011-01-05,Daniel Burke,Echirolles,France,Central,Home Office,2011-01-07,Priority,Auvergne-Rhône-Alpes,2011-01


track repeated purchases, for montioring or forcast trends. 
- Is there any high churn? 
- Is the customer engage more over the time? to see if the higher frequenctly the longer the customer stay with you.